<h1 style='color:orange; text-align:center; font-weight:bold'>Semantic Similarity</h1>

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.nn.functional import cosine_similarity
import matplotlib.pyplot as plt
import warnings
import umap
import torch.nn.functional as F
import pandas as pd

In [24]:
# surpress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [25]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [26]:
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

## **2.1 Semantic Similarity: Word Level**

In [30]:
word_pairs = [
    ("ayah", "bapak"),
    ("ayah", "kursi"),
    ("ayah", "ponsel"),
    ("ayah", "guru"),
    ("ayah", "jeruk"),
    ("ayah", "pekerjaan"),
    ("ayah", "metamorfosis")
]

In [31]:
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        # take the first actual token (0: [CLS])
        return outputs.last_hidden_state[0][1]

similarities = []
for w1, w2 in word_pairs:
    emb1 = get_word_embedding(w1)
    emb2 = get_word_embedding(w2)
    sim = cosine_similarity(emb1, emb2, dim=0).item()
    similarities.append((w1, w2, sim))

In [32]:
for w1, w2, score in similarities:
    print(f"{w1} <> {w2} = {score:.4f}")

ayah <> bapak = 0.7813
ayah <> kursi = 0.6375
ayah <> ponsel = 0.6325
ayah <> guru = 0.7468
ayah <> jeruk = 0.6337
ayah <> pekerjaan = 0.6396
ayah <> metamorfosis = 0.4039


## **2.2 Semantic Similarity: Sentence Level**

In [17]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1)

In [18]:
text1 = "Anak itu sedang membaca buku di perpustakaan."
text2 = "Seorang siswa membaca sebuah buku di dalam perpustakaan."

emb1 = get_embedding(text1)
emb2 = get_embedding(text2)

similarity = cosine_similarity(emb1, emb2)
print(f"Similarity: {similarity.item():.4f}")

Similarity: 0.8338


In [19]:
texts = [
    "Ayah makan nasi.",
    "Bapak menyantap nasi.",
    "Ibu memasak di dapur.",
    "Siswa belajar di kelas.",
    "Dia membaca buku.",
    "Anak itu sedang membaca buku di perpustakaan.",
    "Anak bermain di taman.",
    "Petani menanam padi di sawah.",
    "Burung terbang di langit.",
    "Dokter memeriksa pasien.",
    "Mobil melaju di jalan raya.",
    "Guru mengajar matematika.",
    "Saya menonton film di bioskop.",
    "Perempuan itu membeli sayur di pasar.",
    "Kami berjalan-jalan di pusat kota.",
    "Polisi mengatur lalu lintas.",
    "Orang-orang antre di halte bus.",
    "Seorang siswa membaca sebuah buku di dalam perpustakaan.",
    "Dia menulis surat untuk temannya.",
    "Mereka berolahraga di lapangan."
]

embeddings = [get_embedding(t) for t in texts]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [47]:
n = len(embeddings)
sim_matrix = torch.zeros((n, n))

for i in range(n):
    for j in range(n):
        sim_matrix[i][j] = F.cosine_similarity(embeddings[i], embeddings[j])

for i, text in enumerate(texts):
    print(f"sent{i+1:02}: {text}")

# change to df
labels = [f"sent{i+1:02}" for i in range(n)]
df_sim = pd.DataFrame(sim_matrix.numpy(), index=labels, columns=labels)
display(df_sim.round(3).style.background_gradient(cmap='YlOrRd'))

sent01: Ayah makan nasi.
sent02: Bapak menyantap nasi.
sent03: Ibu memasak di dapur.
sent04: Siswa belajar di kelas.
sent05: Dia membaca buku.
sent06: Anak itu sedang membaca buku di perpustakaan.
sent07: Anak bermain di taman.
sent08: Petani menanam padi di sawah.
sent09: Burung terbang di langit.
sent10: Dokter memeriksa pasien.
sent11: Mobil melaju di jalan raya.
sent12: Guru mengajar matematika.
sent13: Saya menonton film di bioskop.
sent14: Perempuan itu membeli sayur di pasar.
sent15: Kami berjalan-jalan di pusat kota.
sent16: Polisi mengatur lalu lintas.
sent17: Orang-orang antre di halte bus.
sent18: Seorang siswa membaca sebuah buku di dalam perpustakaan.
sent19: Dia menulis surat untuk temannya.
sent20: Mereka berolahraga di lapangan.


,sent01,sent02,sent03,sent04,sent05,sent06,sent07,sent08,sent09,sent10,sent11,sent12,sent13,sent14,sent15,sent16,sent17,sent18,sent19,sent20
sent01,1.000000,0.915000,0.771000,0.582000,0.650000,0.573000,0.659000,0.538000,0.632000,0.649000,0.539000,0.688000,0.623000,0.650000,0.521000,0.605000,0.508000,0.475000,0.605000,0.574000
sent02,0.915000,1.000000,0.758000,0.593000,0.680000,0.608000,0.640000,0.554000,0.635000,0.684000,0.575000,0.716000,0.624000,0.667000,0.538000,0.615000,0.537000,0.508000,0.597000,0.595000
sent03,0.771000,0.758000,1.000000,0.679000,0.526000,0.566000,0.799000,0.702000,0.682000,0.586000,0.637000,0.700000,0.649000,0.665000,0.499000,0.615000,0.538000,0.506000,0.538000,0.558000
sent04,0.582000,0.593000,0.679000,1.000000,0.541000,0.560000,0.759000,0.635000,0.607000,0.550000,0.584000,0.779000,0.643000,0.537000,0.485000,0.613000,0.538000,0.640000,0.489000,0.591000
sent05,0.650000,0.680000,0.526000,0.541000,1.000000,0.771000,0.550000,0.467000,0.552000,0.693000,0.513000,0.632000,0.666000,0.676000,0.590000,0.558000,0.433000,0.678000,0.767000,0.690000
sent06,0.573000,0.608000,0.566000,0.560000,0.771000,1.000000,0.604000,0.472000,0.504000,0.617000,0.547000,0.515000,0.636000,0.758000,0.579000,0.509000,0.516000,0.834000,0.672000,0.594000
sent07,0.659000,0.640000,0.799000,0.759000,0.550000,0.604000,1.000000,0.675000,0.743000,0.511000,0.655000,0.735000,0.682000,0.573000,0.553000,0.644000,0.551000,0.555000,0.540000,0.613000
sent08,0.538000,0.554000,0.702000,0.635000,0.467000,0.472000,0.675000,1.000000,0.644000,0.475000,0.618000,0.627000,0.597000,0.595000,0.477000,0.560000,0.529000,0.487000,0.486000,0.543000
sent09,0.632000,0.635000,0.682000,0.607000,0.552000,0.504000,0.743000,0.644000,1.000000,0.521000,0.634000,0.665000,0.638000,0.534000,0.525000,0.601000,0.519000,0.471000,0.510000,0.622000
sent10,0.649000,0.684000,0.586000,0.550000,0.693000,0.617000,0.511000,0.475000,0.521000,1.000000,0.515000,0.625000,0.555000,0.607000,0.538000,0.605000,0.448000,0.530000,0.621000,0.590000
